# Notebook para manutenção do código

## Home

In [1]:
import logging
import numpy as np
import pandas as pd
from PIL import Image
import streamlit as st

In [2]:
# Extra script
uploaded_files = [
    '../data/kenji/movimentacao-2019.xlsx',
    '../data/kenji/movimentacao-2020.xlsx',
    '../data/kenji/movimentacao-2021.xlsx',
    '../data/kenji/movimentacao-2022.xlsx',
]

In [3]:
def data_ingestion(uploaded_files):
    df = pd.DataFrame()
    for i in uploaded_files:
        df_temp = pd.read_excel(i, engine='openpyxl')
        df = pd.concat([df, df_temp], axis=0, ignore_index=True)
    
    df.drop_duplicates(keep='last', inplace=True)        
    return df
    
# Teste
df_data_ingestion = data_ingestion(uploaded_files)
df_data_ingestion.head(2)

c:\Users\Kenji\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Kenji\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Kenji\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
0,Credito,17/12/2019,Transferência - Liquidação,CEBR3 - CIA ENERGETICA DE BRASILIA,MODAL DTVM LTDA,100.0,53.69,5369.0
1,Credito,17/12/2019,Transferência - Liquidação,ITSA4 - ITAUSA S/A,MODAL DTVM LTDA,100.0,13.6,1360.0


In [51]:
def data_treatment(df):
    # Datatype
    dict_dtype = {'credito_ou_debito': str,
                  'data': str,
                  'tp_movimento': str,
                  'ativo': str,
                  'instituicao': str,
                  'qt_abs': float,
                  'preco_mov': float,
                  'vl_total_abs': float}
    
    df.columns = list(dict_dtype.keys())
    df['preco_mov'].replace('-', 0, inplace=True)
    df['vl_total_abs'].replace('-', 0, inplace=True)
    df = df.astype(dict_dtype)
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

    # (geral) Nova variável: classificação do ativo.
    df['tp_ativo'] = np.select(
        [
            (df['ativo'].str.upper()).str.contains('TESOURO'),
            df['ativo'].str.split(' - ', 0).str[0].str.len() == 5,
            df['ativo'].str.split(' - ', 0).str[0].str.len() == 6,
            df['ativo'].str.contains('DEB'),
            df['ativo'].str.contains('|'.join(['CDB', 'RDB', 'LCA', 'LCI']))
        ],
        [
            'Tipo 1: tesouro',
            'Tipo 2: ações',
            'Tipo 3: BDR',
            'Tipo 4: debêntures',
            'Tipo 5: renda fixa privada'
        ],'?'
    )

    # (geral) Nova variável: ticker.
    df['ticker'] = np.select(
        [
            df['tp_ativo'] == 'Tipo 4: debêntures',
            df['tp_ativo'] == 'Tipo 5: renda fixa privada'
        ],
        [   
            df['ativo'].str[5:12],
            df['ativo'].str[5:17]
        ], df['ativo'].str.split(' - ').str[0]
    )

    # (bolsa) Ajuste específico de ações: zerar a quantidade de compra/venda em caso de dividendos e juros sobre capital próprio.
    condition = df['tp_movimento'].isin(['Transferência - Liquidação', 'Bonificação em Ativos', 'Desdobro'])
    #            df['tp_ativo'].isin(['Tipo 2: ações', 'Tipo 3: BDR'])
    df['qt_abs'] = np.where(~condition, df['qt_abs'], 0)

    # (geral) Nova variável: variação na quantidade de ativos.
    #df['qt'] = df['qt_abs'] * np.where(df['credito_ou_debito'] == 'Credito', 1, -1)

    # (geral) Nova variável: variação na quantidade no valor total.
    df['vl_total'] = df['vl_total_abs'] * np.where(df['credito_ou_debito'] == 'Credito', 1, -1)

    # (bolsa) Nova variável: flag se a negociação é um provento (dividendo, juros sobre capital próprio ou leilão)
    df['evento'] = np.select(
        [
            df['tp_movimento'].isin(['Dividendo', 'Juros Sobre Capital Próprio', 'Fração em Ativos', 'Leilão de Fração']),
            df['tp_movimento'].isin(['Bonificação em Ativos']),
            df['tp_movimento'].isin(['Desdobro', 'Grupamento', ]),
            df['tp_movimento'].isin(['Transferência - Liquidação', 'Compra', 'Venda', 'COMPRA / VENDA', 'COMPRA/VENDA DEFINITIVA/CESSAO']),
            df['tp_movimento'].isin(['Cobrança de Taxa Semestral', 'Atualização'])
        ],
        [
            'dividendo_ou_jcp',
            'bonificacao',
            'split',
            'compra_ou_venda',
            'outros'
        ],
        '?')
    return df

# Teste
df_data_treatment = data_treatment(df_data_ingestion)
df_data_treatment.head(2)

,credito_ou_debito,data,tp_movimento,ativo,instituicao,qt_abs,preco_mov,vl_total_abs,tp_ativo,ticker,vl_total,evento
0,Credito,2019-12-17,Transferência - Liquidação,CEBR3 - CIA ENERGETICA DE BRASILIA,MODAL DTVM LTDA,100.0,53.69,5369.0,Tipo 2: ações,CEBR3,5369.0,compra_ou_venda
1,Credito,2019-12-17,Transferência - Liquidação,ITSA4 - ITAUSA S/A,MODAL DTVM LTDA,100.0,13.60,1360.0,Tipo 2: ações,ITSA4,1360.0,compra_ou_venda


In [50]:
#df_data_treatment['qt_abs'].max()
#df_data_treatment['qt'].max()
df_data_treatment.loc[df_data_treatment['ativo'] == 'Tesouro Selic 2024']

,credito_ou_debito,data,tp_movimento,ativo,instituicao,qt_abs,preco_mov,vl_total_abs,tp_ativo,ticker,vl_total,evento
115,Credito,2021-08-17,Compra,Tesouro Selic 2024,XP INVESTIMENTOS CCTVM S/A,0.0,10900.11,7303.07,Tipo 1: tesouro,Tesouro Selic 2024,7303.07,compra_ou_venda
120,Credito,2021-07-22,Compra,Tesouro Selic 2024,XP INVESTIMENTOS CCTVM S/A,0.0,10852.75,4124.05,Tipo 1: tesouro,Tesouro Selic 2024,4124.05,compra_ou_venda
202,Debito,2022-07-01,Cobrança de Taxa Semestral,Tesouro Selic 2024,XP INVESTIMENTOS CCTVM S/A,0.0,0.00,3.41,Tipo 1: tesouro,Tesouro Selic 2024,-3.41,outros
233,Debito,2022-02-24,Venda,Tesouro Selic 2024,XP INVESTIMENTOS CCTVM S/A,0.0,11379.60,10014.05,Tipo 1: tesouro,Tesouro Selic 2024,-10014.05,compra_ou_venda
234,Credito,2022-02-17,Compra,Tesouro Selic 2024,XP INVESTIMENTOS CCTVM S/A,0.0,11354.11,2270.82,Tipo 1: tesouro,Tesouro Selic 2024,2270.82,compra_ou_venda
235,Credito,2022-02-02,Compra,Tesouro Selic 2024,XP INVESTIMENTOS CCTVM S/A,0.0,11301.17,1130.12,Tipo 1: tesouro,Tesouro Selic 2024,1130.12,compra_ou_venda
237,Credito,2022-01-20,Compra,Tesouro Selic 2024,XP INVESTIMENTOS CCTVM S/A,0.0,11259.53,1576.33,Tipo 1: tesouro,Tesouro Selic 2024,1576.33,compra_ou_venda


In [32]:
def only_tesouro(df):
    df = df.loc[df['tp_ativo'] == 'Tipo 1: tesouro']
    df = df.groupby(['tp_ativo','ticker','data']).agg({'qt':'sum', 'vl_total':'sum'}).reset_index()
    df['preco_mov'] = np.where(df['qt'] != 0, round(df['vl_total'] / df['qt'], 2), 0)
    df = df.loc[df['tp_ativo'] == 'Tipo 1: tesouro'].sort_values(by=['ticker','data'], ascending=True)
    df = df[['data', 'ticker', 'qt', 'preco_mov', 'vl_total']]
    return df

# Teste
df_tesouro = only_tesouro(df_data_treatment)
df_tesouro.head(2)

,data,ticker,qt,preco_mov,vl_total
0,2021-06-30,Tesouro IPCA+ 2035,0.0,0.0,3476.79
1,2022-07-01,Tesouro IPCA+ 2035,0.0,0.0,-3.14


In [7]:
def only_bolsa(df):
    df = df.loc[(df['tp_ativo'] == 'Tipo 2: ações') | (df['tp_ativo'] == 'Tipo 3: BDR')].sort_values(by=['ticker','data'], ascending=True)
    df = df[['data', 'ticker', 'tp_movimento', 'evento', 'qt', 'preco_mov', 'vl_total']]
    df = df.groupby(['data', 'ticker','evento']).agg({'qt':'sum', 'vl_total':'sum'}).reset_index(drop=False)
    return df

df_bolsa = only_bolsa(df_data_treatment)
df_bolsa.head()

,data,ticker,evento,qt,vl_total
0,2019-11-05,BPAN4,compra_ou_venda,600.0,5502.0
1,2019-11-06,MGLU3,compra_ou_venda,-100.0,-4499.0
2,2019-11-08,BPAN4,compra_ou_venda,-600.0,-5598.0
3,2019-11-08,CCRO3,compra_ou_venda,300.0,5046.0
4,2019-11-12,CYRE3,compra_ou_venda,100.0,2725.0


## Tesouro Direto

In [8]:
import pandas as pd
import yfinance as yf
from lib.history import tesouro

### Parte 1: Tabela Dinâmica

In [9]:
df_tesouro.head()

,data,ticker,qt,preco_mov,vl_total
0,2021-06-30,Tesouro IPCA+ 2035,0.0,0.0,3476.79
1,2022-07-01,Tesouro IPCA+ 2035,0.0,0.0,-3.14
2,2021-11-04,Tesouro IPCA+ 2045,0.0,0.0,3530.80
3,2022-07-01,Tesouro IPCA+ 2045,0.0,0.0,-3.48
4,2020-07-10,Tesouro Prefixado 2023,0.0,0.0,1472.00


In [10]:
# Carregando os preços diários extraídos da API do Tesouro.
df_history_tesouro = tesouro()
df_history_tesouro.head(2)

,data,ticker,preco_hist
0,2021-01-29,Tesouro IPCA+ com Juros Semestrais 2045,4505.79
1,2021-01-29,Tesouro IPCA+ com Juros Semestrais 2024,3995.84


In [21]:
#df = pd.merge(df_history_tesouro, df_tesouro, on=['data','ticker'], how='left').fillna(0).sort_values(['ticker','data'])
#df.info()
#df.loc[df['qt'] > 0]#, 'ticker'].unique()
#df.head()
#df['qt'] == 0
df_tesouro['qt'] > 0

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
Name: qt, dtype: bool

In [12]:
df['data'].min()

Timestamp('2004-12-31 00:00:00')

In [ ]:
df_historic = 
df_tesouro = 

# Merge entre preços históricos e investimentos do B3.
list_ticker = df_tesouro['ticker'].unique()
dt_min = df_tesouro['data'].min()

df_historic = df_historic.loc[(df_historic['ticker'].isin(list_ticker)) &
                              (df_historic['data'] >= dt_min)].sort_values(by=['ticker','data'], ascending=True)

df_tesouro_hist = pd.merge(df_historic, df_tesouro, on=['data','ticker'], how='left').fillna(0)

# Cálculo do valor acumulado para cada ticker.
for i in list_ticker:
    df_tesouro_hist.loc[df_tesouro_hist['ticker'] == i, 'qt_acum'] = df_tesouro_hist.loc[df_tesouro_hist['ticker'] == i, 'qt'].cumsum(skipna=True)

    # Condição: excluir datas sem movimentação ou que não havia tesouro na carteira.
    df_tesouro_hist = df_tesouro_hist.loc[(df_tesouro_hist['qt'] != 0) | (df_tesouro_hist['qt_acum'] != 0)]
    df_tesouro_hist['vl_atualizado'] = (df_tesouro_hist['qt_acum'] * df_tesouro_hist['preco_hist']).round(2)


In [ ]:
# Juntando os dados do usuário com os dados da API.
df_tesouro_historico = lib.merge_historic_tesouro(df_hist_tesouro, df_tesouro)
df_tesouro_historico.head()

In [ ]:
# Passo extra para criar uma coluna identificando o último dia do mês de cada ativo (útil para os plots)
df_tesouro_historico = lib.create_column_last_day(df_tesouro_historico)
df_tesouro_historico.head()

In [ ]:
# Selecionar os tickers a serem visualizados (desnecessário neste notebook)
'''
# Filtro de tickers.
list_ticker = st.multiselect('Escolha o(s) investimento(s):',
                                df_tesouro_historico['ticker'].unique().tolist(),
                                df_tesouro_historico['ticker'].unique().tolist())
df_tesouro_historico = df_tesouro_historico.loc[df_tesouro_historico['ticker'].isin(list_ticker)]
'''

In [ ]:
# Preparar os dados exclusivamente para a Tabela Dinâmica (visão 1)
df_plot = df_tesouro_historico.loc[df_tesouro_historico['dummy_ultimo_dia'] == 1]
tab1, data_col = lib.custom_pivot_table(df_plot, col_value='vl_atualizado')
tab1

### Parte 2: Gráfico de Linha com Benchmark

In [ ]:
# Extração das variações dos benchmarks (API do BC e yfinance)
df_hist_bench = etl_benchmark_historic_price()
df_hist_bench.head()

In [ ]:
# Junção dos dados inputados pelo usuário e os dados históricos do benchmarks
df_tesouro_historico_agg = lib.merge_historic_benchmark(df_tesouro_historico, df_hist_bench)
df_tesouro_historico_agg.head()

In [ ]:
# Arrumando dados em um formato melhor para o gráfico.
tab2 = lib.custom_data_lineplot(df_tesouro_historico_agg, ['ibov', 'sp500', 'cdi', 'ipca'])
tab2.head()

### Parte 3: KPI

In [ ]:
date_interval = (datetime.datetime(2022, 7, 3), datetime.datetime(2022, 12, 1))
date_interval

In [ ]:
df_date = pd.DataFrame({'data': pd.date_range(df_tesouro_historico['data'].min().to_pydatetime() - datetime.timedelta(days=1),
                                                df_tesouro_historico['data'].max().to_pydatetime())})
df_kpi = pd.merge(df_date, df_tesouro_historico, on='data', how='left')
df_kpi = df_kpi.groupby('data').agg({'qt':'sum', 'qt_acum':'sum',  'vl_atualizado':'sum'}).reset_index()
df_kpi['vl_atualizado'] = np.where(df_kpi['qt_acum'] != 0, df_kpi['vl_atualizado'], np.nan)
df_kpi['vl_atualizado'] = df_kpi['vl_atualizado'].fillna(method='ffill').fillna(0)
df_kpi.head()

In [ ]:
# Cálculo dos aportes históricos.
vl_aporte = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) &
                                        (df_tesouro_historico['vl_total'] > 0) &
                                        (df_tesouro_historico['data'] <= date_interval[1]), 'vl_total'].sum()

vl_aporte_delta = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) &
                                            (df_tesouro_historico['vl_total'] > 0) &
                                            (df_tesouro_historico['data'].between(date_interval[0], date_interval[1])), 'vl_total'].sum()

# Cálculo dos valores resgatados.
vl_resgate = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) &
                                        (df_tesouro_historico['vl_total'] < 0) &
                                        (df_tesouro_historico['data'] <= date_interval[1]), 'vl_total'].sum()

vl_resgate_delta = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) &
                                            (df_tesouro_historico['vl_total'] < 0) &
                                            (df_tesouro_historico['data'].between(date_interval[0], date_interval[1])), 'vl_total'].sum()

# Cálculo do valor patrimonial.
vl_patrimonio = df_kpi.loc[df_kpi['data'] == date_interval[1], 'vl_atualizado'].sum()
vl_patrimonio_delta = vl_patrimonio - df_kpi.loc[df_kpi['data'] == date_interval[0], 'vl_atualizado'].sum()

# Cálculo do rendimento.
rendimento_nominal = round((vl_patrimonio - vl_resgate - vl_aporte) / vl_aporte * 100 , 1)
if  vl_aporte_delta != 0:
    rendimento_nominal_delta = round((vl_patrimonio_delta - vl_resgate_delta - vl_aporte_delta) / vl_aporte_delta * 100 , 1)
else:
    rendimento_nominal_delta = 0

vl_aporte, vl_aporte_delta, vl_resgate, vl_resgate_delta, vl_patrimonio, vl_patrimonio_delta, rendimento_nominal, rendimento_nominal_delta

## Bolsa

In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
df_bolsa['data'] = pd.to_datetime(df_bolsa['data'], format='%Y-%m-%d')
df_bolsa.head()

In [ ]:
# Salvar lá encima!
def etl_bolsa_historic_price(list_ticker_b3: list, start_date: str, end_date: str) -> np.array:

    # Utilizando a api do yf
    list_ticker_yf = [i + '.SA' for i in list_ticker_b3]
    long_string = ' '.join(list_ticker_yf)
    yf_df = yf.download(long_string, start=start_date, end=end_date, group_by='column', actions=True, interval='1d')
    
    # Obter o preço histórico e os eventos de agrupamento/desdobramento de ações
    df_price = yf_df['Close'].reset_index().sort_values('Date', ascending=True).round(2)#.fillna(method='ffill')

    # Ajustes gerais na base
    df_price.columns = ['data'] + list(list_ticker_b3)    
    df_price['data'] = pd.to_datetime(df_price['data'])
    df_price = pd.melt(df_price, id_vars=['data'], value_vars=list(list_ticker_b3), var_name='ticker', value_name='preco')
    
    return df_price

In [ ]:
# Passo 1: extrair dados atualizados de cada ticker.
df_bolsa_historico = etl_bolsa_historic_price(list_ticker_b3=df_bolsa['ticker'].sort_values().unique(),
                                              start_date=df_bolsa['data'].min(),
                                              end_date=df_bolsa['data'].max())
df_bolsa_historico.head()

In [ ]:
"""
# Experimentando como usar o stock split
start_date = df_bolsa.loc[df_bolsa['ticker'] == 'B3SA3', 'data'].dt.date.min()
end_date = df_bolsa.loc[df_bolsa['ticker'] == 'B3SA3', 'data'].dt.date.max()
long_string = 'B3SA3.SA ITSA4.SA'

#start_date
yf_df = yf.download(long_string, start=start_date, end=end_date, group_by='column', actions=True, interval='1wk')
yf_df_close = yf_df['Close'].reset_index().sort_values('Date', ascending=False).round(2).fillna(method='ffill')
yf_df_split = yf_df['Stock Splits'].reset_index().sort_values('Date', ascending=False).cumsum().round(2).fillna(method='ffill').replace(0, 1)

display(df_bolsa.loc[df_bolsa['ticker'] == 'B3SA3'])
#display(yf_df)
display(yf_df_close.sort_values('Date'))
display(yf_df_split.sort_index())

for i in ['B3SA3.SA']:
    yf_df_close[i] = yf_df_close[i] * (yf_df_split[i])

df_price = yf_df_close.sort_values('Date', ascending=True)
display(df_price)
"""

In [ ]:
# Passo 2: juntar a carteira com os preços históricos.
df_bolsa2 = pd.merge(df_bolsa_historico, df_bolsa, on=['data', 'ticker'], how='left').fillna(0).sort_values(['ticker', 'data'])
df_bolsa2.head()

In [ ]:
# Passo 3: calcular a quantidade acumulada.
for i in df_bolsa2['ticker'].sort_values().unique():
    df_bolsa2.loc[df_bolsa2['ticker'] == i, 'qt_acum'] = df_bolsa2.loc[df_bolsa2['ticker'] == i, 'qt'].cumsum(skipna=True)
    
df_bolsa_acum = df_bolsa2.loc[(df_bolsa2['qt'] != 0) | (df_bolsa2['qt_acum'] != 0)]

display(df_bolsa_acum.head(2))

In [ ]:
# Passo 4: corrigir preço histórico por agrupamento/desdobramento de ações.
def custom_price_adjustment_for_split(df):
    df = df.sort_values(['ticker', 'data'], ascending=False)
    for i in df['ticker'].unique():
        array_event = df.loc[df['ticker'] == i, 'evento']

        # Caso 1: não tem spit.
        if len(array_event[array_event == 'split']) == 0:
            df.loc[df_bolsa_acum['ticker'] == i, 'preco_fix'] = df.loc[df['ticker'] == i, 'preco']

        # Caso 2: tem split(s).
        else:
            list_index = array_event[array_event == 'split'].index.tolist()
            for j in list_index:
                df.loc[j-1, 'multiplicador_split'] = df.loc[j, 'qt_acum'] / df.loc[j - 1, 'qt_acum']
        df.loc[df['ticker'] == i, 'multiplicador_split'] = df.loc[df['ticker'] == i, 'multiplicador_split'].fillna(0).cumsum()

    # Corrigindo os preços por splits.
    df['multiplicador_split'] = df['multiplicador_split'].replace(0, 1)
    df['preco_fix'] = df['preco'] * df['multiplicador_split']
    df['vl_atualizado_fix'] = df['preco_fix'] * df['qt_acum']
    df = df.sort_values(['ticker', 'data'], ascending=True)
    return df

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    display(df_preco_fix)

In [ ]:
df_bolsa_split = custom_price_adjustment_for_split(df_bolsa_acum)
df_bolsa_split.head()

In [ ]:
# Passo 5: criar uma coluna identificando o último dia do mês de cada ativo (útil para os plots)
df_bolsa_split = lib.create_column_last_day(df_bolsa_split)
df_bolsa_split.head()

In [ ]:
# Passo 6: criar tabela dinâmica customizada
df_plot = df_bolsa_split.loc[df_bolsa_split['dummy_ultimo_dia'] == 1]
df_plot, data_col = lib.custom_pivot_table(df_plot, col_value='vl_atualizado_fix')

df_plot.head()

-----------

Print melhorado!

```python
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)
```

In [ ]:
# Teste
a = ['a', 'b', 'c']
for index, value in enumerate(a):
    print(index, value)

In [ ]:
a = ['a', 'b', 'c']
b = [4, 5, 6]    
for value_a, value_b in zip(a, b):
    print(value_a, value_b)